In [ ]:
import sqlite3
import pandas as pd
import os

# Path to your database
db_path = 'data/air_quality.db'

# Ensure the data directory exists
os.makedirs(os.path.dirname(db_path), exist_ok=True)


In [ ]:
# Load the CSV
df = pd.read_csv('data/pollution_us_2000_2016.csv')

# Create/connect to the database (creates file if it doesn't exist)
conn = sqlite3.connect(db_path)

# Write DataFrame to SQL table
df.to_sql("air_quality_measurements", conn, if_exists="replace", index=False)

conn.close()

In [22]:
df.head()

,Unnamed: 0,State Code,County Code,Site Num,Address,State,County,City,Date Local,NO2 Units,...,SO2 Units,SO2 Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,CO Units,CO Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI
0,0,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,3.000000,9.0,21,13.0,Parts per million,1.145833,4.2,21,NaN
1,1,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,3.000000,9.0,21,13.0,Parts per million,0.878947,2.2,23,25.0
2,2,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,2.975000,6.6,23,NaN,Parts per million,1.145833,4.2,21,NaN
3,3,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,2.975000,6.6,23,NaN,Parts per million,0.878947,2.2,23,25.0
4,4,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-02,Parts per billion,...,Parts per billion,1.958333,3.0,22,4.0,Parts per million,0.850000,1.6,23,NaN


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1746661 entries, 0 to 1746660
Data columns (total 29 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Unnamed: 0         int64  
 1   State Code         int64  
 2   County Code        int64  
 3   Site Num           int64  
 4   Address            object 
 5   State              object 
 6   County             object 
 7   City               object 
 8   Date Local         object 
 9   NO2 Units          object 
 10  NO2 Mean           float64
 11  NO2 1st Max Value  float64
 12  NO2 1st Max Hour   int64  
 13  NO2 AQI            int64  
 14  O3 Units           object 
 15  O3 Mean            float64
 16  O3 1st Max Value   float64
 17  O3 1st Max Hour    int64  
 18  O3 AQI             int64  
 19  SO2 Units          object 
 20  SO2 Mean           float64
 21  SO2 1st Max Value  float64
 22  SO2 1st Max Hour   int64  
 23  SO2 AQI            float64
 24  CO Units           object 
 25  CO Mean           

In [41]:
df.shape

(1746661, 29)

In [24]:
print(df.isnull().sum())

Unnamed: 0                0
State Code                0
County Code               0
Site Num                  0
Address                   0
State                     0
County                    0
City                      0
Date Local                0
NO2 Units                 0
NO2 Mean                  0
NO2 1st Max Value         0
NO2 1st Max Hour          0
NO2 AQI                   0
O3 Units                  0
O3 Mean                   0
O3 1st Max Value          0
O3 1st Max Hour           0
O3 AQI                    0
SO2 Units                 0
SO2 Mean                  0
SO2 1st Max Value         0
SO2 1st Max Hour          0
SO2 AQI              872907
CO Units                  0
CO Mean                   0
CO 1st Max Value          0
CO 1st Max Hour           0
CO AQI               873323
dtype: int64


In [26]:
cols = ['State','City','Date Local','NO2 AQI','O3 AQI','SO2 AQI','CO AQI']

In [28]:
df_ = df[cols]

In [30]:
df_.head(10)

,State,City,Date Local,NO2 AQI,O3 AQI,SO2 AQI,CO AQI
0,Arizona,Phoenix,2000-01-01,46,34,13.0,NaN
1,Arizona,Phoenix,2000-01-01,46,34,13.0,25.0
2,Arizona,Phoenix,2000-01-01,46,34,NaN,NaN
3,Arizona,Phoenix,2000-01-01,46,34,NaN,25.0
4,Arizona,Phoenix,2000-01-02,34,27,4.0,NaN
5,Arizona,Phoenix,2000-01-02,34,27,4.0,26.0
6,Arizona,Phoenix,2000-01-02,34,27,NaN,NaN
7,Arizona,Phoenix,2000-01-02,34,27,NaN,26.0
8,Arizona,Phoenix,2000-01-03,48,14,16.0,NaN
9,Arizona,Phoenix,2000-01-03,48,14,16.0,28.0


In [31]:
df_cleaned = df_.groupby(['State', 'City', 'Date Local'], as_index=False).agg({
    'NO2 AQI': 'max',
    'O3 AQI': 'max',
    'SO2 AQI': 'max',
    'CO AQI': 'max'})

,State,City,Date Local,NO2 AQI,O3 AQI,SO2 AQI,CO AQI
0,Alabama,Birmingham,2013-12-01,37,24,1.0,6.0
1,Alabama,Birmingham,2013-12-02,30,12,3.0,6.0
2,Alabama,Birmingham,2013-12-03,21,11,3.0,3.0
3,Alabama,Birmingham,2013-12-04,18,13,1.0,2.0
4,Alabama,Birmingham,2013-12-05,15,13,1.0,2.0


In [37]:
df_cleaned['Date Local'] = pd.to_datetime(df_cleaned['Date Local'])
df_cleaned = df_cleaned.sort_values(by=['Date Local','State']).reset_index(drop=True)
df_cleaned.head()

,State,City,Date Local,NO2 AQI,O3 AQI,SO2 AQI,CO AQI
0,Arizona,Tucson,2000-01-01,36,33,6.0,8.0
1,Arizona,Phoenix,2000-01-01,46,34,13.0,25.0
2,Arizona,Scottsdale,2000-01-01,101,33,3.0,15.0
3,California,Vandenberg Air Force Base,2000-01-01,1,36,1.0,1.0
4,California,Vallejo,2000-01-01,25,21,4.0,6.0


In [40]:
df_cleaned.shape

(384649, 7)

In [42]:
df_cleaned.to_csv("/home/gabijajulio/air-quality-forecasting/data/cleaned_pollutants_sorted.csv", index=False)